# Проект "Исследование рынка заведений общественного питания Москвы" #
Цель проекта:
- на основании исследования рынка заведений общественного питания в Москве, подготовить презентацию для привлечения инвестиций в открытие кафе в Москве

# Cодержание #

<a id='data_description'></a>
# Описание данных #
Таблица rest_data:
id — идентификатор объекта;
object_name — название объекта общественного питания;
chain — сетевой ресторан;
object_type — тип объекта общественного питания;
address — адрес;
number — количество посадочных мест.

<a id='analyse_plan'></a>
# План проведения исследования #
1. загрузка данных
2. подготовка данных
3. анализ данных

<a id='import'></a>
# Импорт библиотек #

In [1]:
import io

import pandas as pd
from IPython.display import display
import numpy as np
from scipy import stats as st
import plotly_express as px
import os
import warnings
import requests

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

<a id='data'></a>
# Загрузка данных #

In [2]:
path_1 = 'datasets/'
path_2 = '/datasets/'
rest_data = 'rest_data.csv'
street_reg_data = 'mosgaz-streets.csv'
#region_data = 'moscow_region.csv'
# street_data = 'street_data.csv'
# region_data = 'region_data.csv'
path_rest = ''
path_street_reg = ''
# path_street = ''
# path_region = ''

if os.path.exists(path_1):
    path_rest = path_1 + rest_data
    path_street_reg = path_1 + street_reg_data
    # path_street = path_1 + street_data
    # path_region = path_1 + region_data
elif os.path.exists(path_2):
    path_rest = path_2 + rest_data
    path_street_reg = path_2 + street_reg_data
    # path_street = path_2 + street_data
    # path_region = path_2 + region_data
else: print('Данные отсутствуют. Проверьте путь к папкам с данными')

data = pd.read_csv(filepath_or_buffer=path_rest)
data_street_reg = pd.read_csv(filepath_or_buffer=path_street_reg)
# data_street = pd.read_csv(filepath_or_buffer=path_street, sep=';')
# data_region = pd.read_csv(filepath_or_buffer=path_region, sep=';')

In [3]:
data_street_reg

,streetname,areaid,okrug,area
0,Выставочный переулок,17,ЦАО,Пресненский район
1,улица Гашека,17,ЦАО,Пресненский район
2,Большая Никитская улица,17,ЦАО,Пресненский район
3,Глубокий переулок,17,ЦАО,Пресненский район
4,Большой Гнездниковский переулок,17,ЦАО,Пресненский район
...,...,...,...,...
4393,Вознесенский проезд,17,ЦАО,Пресненский район
4394,Волков переулок,17,ЦАО,Пресненский район
4395,Поварская улица,17,ЦАО,Пресненский район
4396,Кудринская площадь,17,ЦАО,Пресненский район


In [4]:
data_street_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   streetname  4398 non-null   object
 1   areaid      4398 non-null   int64 
 2   okrug       4398 non-null   object
 3   area        4398 non-null   object
dtypes: int64(1), object(3)
memory usage: 137.6+ KB


In [5]:
# data_region.info()

In [6]:
# data_street.info()

In [7]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB
None


In [8]:
print(data.chain.unique())
print(data.object_type.unique())

['нет' 'да']
['кафе' 'столовая' 'закусочная' 'предприятие быстрого обслуживания'
 'ресторан' 'кафетерий' 'буфет' 'бар' 'магазин (отдел кулинарии)']


## Итоги предварительного осмотра данных ##
1. data['object_name']:
   - перевести данные в нижний регистр
2. data['chain'].dtype(object) -> data['chain'].dtype(category)
3. data['object_type'].dtype(object) -> data['object_type'].dtype(category)
4. data['address']:
   - перевести в нижний регистр
   - выделить отдельный столбец с названием улицы
5. перевести в нижний регистр значения:
   - data_street_reg['streetname']
   - data_street_reg['okrug']
   - data_street_reg['area']

<a id='prepare'></a>
# Предобработка данных #

<a id='prepare_columns'></a>
## Переименование столбцов ##

In [9]:
# data_street.columns = data_street.columns.str.strip().str.lower()
# data_region.columns = data_region.columns.str.strip().str.lower()

In [10]:
# data_region = data_region.drop(columns=['global_id', 'latin_name', 'unnamed: 6'])
# data_region = data_region.rename(columns = {'kod' : 'um_te'})
# data_street = data_street.drop(columns=['um_trans', 'um_tm', 'unnamed: 9'])

<a id='prepare_change'></a>
## Изменение данных ##

<a id='prepare_change_object_data'></a>
### Обработка строковых значений ###

In [11]:
# data_street['um_namef'] = data_street['um_namef'].str.strip().str.lower()
# data_street['um_names'] = data_street['um_names'].str.strip().str.lower()
# data_region['name'] = data_region['name'].str.strip().str.lower()
data['object_name'] = data['object_name'].str.strip().str.lower()
data['address'] = data['address'].str.strip().str.lower()
data_street_reg['streetname'] = data_street_reg['streetname'].str.strip().str.lower()
data_street_reg['okrug'] = data_street_reg['okrug'].str.strip().str.lower()
data_street_reg['area'] = data_street_reg['area'].str.strip().str.lower()

In [12]:
print(data.shape)
include_words = ['поселение','город зеленоград','город московский','посёлок','деревня','город щербинка','город троицк']
data = data.drop(index=data[data.stack().str.contains('|'.join(include_words)).any(level=0)].index)
print(data.shape)

(15366, 6)
(14498, 6)


<a id='prepare_change_um_te'></a>
### Подготовка data_street['um_te'] ###

In [13]:
# temp_data = pd.DataFrame(columns=data_street.columns)
# for i, raw in data_street.iterrows():
#     um_te = str.split(raw['um_te'], ';')
#     if len(um_te) > 1:
#         data_street.at[i, 'um_te'] = um_te[0]
#         for _ in range(1, len(um_te)):
#             raw['um_te'] = um_te[_]
#             temp_data = temp_data.append(raw, ignore_index=True)
#
# data_street = data_street.append(temp_data, ignore_index=True)

<a id='prepare_type'></a>
## Изменение типов данных ##

In [14]:

data['chain'] = data['chain'].replace(to_replace=['да', 'нет'], value=['сетевое', 'несетевое']).astype('category')
data['object_type'] = data['object_type'].astype('category')
# data_street['um_te'] = data_street['um_te'].astype('int64')

<a id='prepare_street'></a>
## Создание нового столбца ##

In [15]:
data['street'] = data['address'].apply(lambda x: str.split(x, ',')[1].strip() if 'город' in x else str.split(x, ',')[0].strip())

<a id='prepare_add_region'></a>
## Добавление районов ##

In [20]:
data = data.merge(data_street_reg[['streetname', 'area']], left_on='street', right_on='streetname', how='left')\
           .drop(columns='streetname')
data.info()

(29141, 9)

In [22]:
data = data.drop(index=data[data['area'].isna()].index)
data.shape

(28997, 9)

In [ ]:
# add_data = data_street.merge(data_region[['um_te','name']], on='um_te', how='left', )
# add_data.shape

In [ ]:
# data_feee = data.merge(add_data[['um_namef', 'name']], left_on='street', right_on='um_namef', how='left')\
#     .rename(columns={'name':'district'})\
#     .drop(columns='um_namef').drop_duplicates()
# data_feee[data_feee['district'].isna()]['street'].unique()

In [ ]:
# data_district = data.merge(add_data[['um_namef', 'name']], left_on='street', right_on='um_namef', how='left')\
#     .rename(columns={'name':'district'})\
#     .drop(columns='um_namef').drop_duplicates()\
#     .dropna()

# data_district = data_district.assign(street_coord=0,
#                                      area=0)
# data_district.isna().sum()
# data_district.shape

### Получение районов от Yandex.геокодер ###

In [ ]:
# data_district[data_district['district'].isna()]['address'].nunique()

In [ ]:
# def mapmaker_by_address(address):
#     params = {"format" : "json",
#               "lang"   : "ru_RU",
#               "apikey" : "18868481-6577-401f-9a16-cef4999b3029",
#               "kind"   : "street"}
#     #"kind"   : "district"
#
#     url = 'https://geocode-maps.yandex.ru/1.x/?geocode=' + address
#     #сделаем запрос к геокодеру
#     response = requests.get(url, params=params)
#     raw_response = response.json()
#     try:
#         coord = response.json()['response']["GeoObjectCollection"]["featureMember"][0]['GeoObject']['Point']['pos']
#     except: coord = 'не найден'
#     try:
#         district = response.json()['response']["GeoObjectCollection"]["featureMember"][0]['GeoObject']["metaDataProperty"]["GeocoderMetaData"]["Address"]["Components"][4]["name"]
#     except: district = 'не найден'
#     try:
#         area = response.json()['response']["GeoObjectCollection"]["featureMember"][0]['GeoObject']["metaDataProperty"]["GeocoderMetaData"]["Address"]["Components"][3]["name"]
#     except: area = 'не найден'
#
#     return coord, district, area
#
# address = 'город москва, улица егора абакумова, дом 9'
# #test = mapmaker_by_address(address='ярославский район')
# test = mapmaker_by_address(address=address)

In [ ]:
# for address in data_district.query('district.isna()')['address'].unique():
#     coord, district, area = mapmaker_by_address(address=address)
#
#     data_district.loc[data['address'] == address, ['street_coord']] = coord
#     data_district.loc[data['address'] == address, ['district']] = district
#     data_district.loc[data['address'] == address, ['area']] = area
#
# data_district.to_csv('datasets/data_part_district.csv', sep=',', encoding='utf-8')

<a id='prepare_dup'></a>
## Поиск дубликатов/пропусков данных ##

In [23]:
print(data.isna().sum())
print(data.duplicated().sum())

id             0
object_name    0
chain          0
object_type    0
address        0
number         0
street         0
streetname     0
area           0
dtype: int64
79


In [ ]:
# print(data_district.isna().sum())
# print(data_district.duplicated().sum())

## Итоги предобработки данных ##
1. переведены в нижний регистр
   - data['object_name']
   - data['address']
2. изменен тип данных:
   - data['chain']
   - data['object_type']
3. создан новый столбец:
   - data['street'] - улица города
   - data['area] - район города

<a id='analyse'></a>
# Анализ данных #

<a id='analyse_object_kind'></a>
## Соотношение видов объектов общественного питания ##

In [ ]:
fig = px.pie(data_frame=data.groupby(by='chain', axis='index')\
                            .agg({'id':'count'})\
                            .reset_index()\
                            .rename(columns={'chain':'type',
                                             'id':'count'}),
             names='type',
             values='count',
             template='seaborn',
             title='Отношение количества сетевых/несетевых заведений',
             width=1000, height=600,)
fig.update_layout(legend_title='Вид заведения')
fig.show()


## Итоги осмотра соотношения видов заведений ##
1. на рынке общественного питания Москвы преобладают несетевые заведения.
   Доли распределены следующим образом:
   - несетевые заведения - 80.6%(12050 шт.)
   - сетевые заведения - 19.4%(2896 шт.)

## Соотношение типов объектов общественного питания ##

In [ ]:
fig = px.pie(data_frame=data.groupby(by='object_type', axis='index')\
                            .agg({'id':'count'})\
                            .reset_index()\
                            .rename(columns={'id':'count'}),
             names='object_type',
             values='count',
             template='seaborn',
             title='Соотношение типов заведений',
             width=1000, height=600,)
fig.update_layout(legend_title = 'Тип заведения')
fig.show()

## Итоги осмотра соотношения типов объектов общественного питания ##
1. 83.8% объектов общественного питания составляют следующие типы:
   - кафе - 39.2%(5863 шт.)
   - столовая - 17.1%(2553 шт.)
   - ресторан - 14.9%(2227 шт.)
   - предприятие быстрого обслуживания - 12.6%(1886 шт.)
2. оставшуюся долю - 16.2% составляют следующие типы
   - бар
   - буфет
   - кафетерий
   - закусочная
   - магазин(отдел кулинарии)

<a id='analyse_kind_of_type'></a>
## Виды заведений характерные для типов заведений ##

In [ ]:
fig = px.bar(data_frame=data.pivot_table(index=['object_type', 'chain'],
                                         values=['id'],
                                         aggfunc='count').reset_index(),
       x='object_type',
       y='id',
       color='chain',
       barmode='group',
       template='seaborn',
       text_auto=True,
       labels={'id':'Кол-во заведений',
               'object_type':'Тип заведения',
               'chain':'Вид заведения'},
       title='Виды заведений характерные для типов заведений',
             width=1000, height=600,)
fig.show()

## Итоги осмотра характерных видов заведений для типов заведений ##
1. Сетевой вид заведения наиболее характерен для следующих типов заведений:
   - кафе
   - предприятие быстрого обслуживания
   - ресторан

<a id='analyse_net_seat'></a>
## Характеристика сетевых заведений по количеству посадочных мест ##

In [ ]:
centroid = data['number'].mean()

In [ ]:
data['seat_type'] = data['number'].apply(lambda x: f'много(>={centroid.round()})' if x >= centroid else f'мало(<{centroid.round()})')
data['seat_type'] = data['seat_type'].astype('category')

In [ ]:
fig = px.pie(data_frame=data.query('chain == "сетевое"')
                            .groupby(by='seat_type')
                            .agg({'id':'count'})
                            .reset_index()
                            .rename(columns={'id':'count'}),
             names='seat_type',
             values='count',
             template='seaborn',
             title='Характеристика сетевых заведений по количеству посадочных мест',
             width=1000, height=600,)
fig.update_layout(legend_title = 'Кол-во мест')
fig.show()

## Итоги осмотра характеристики сетевых заведений по количеству посадочных мест ##
Сетевые заведения предпочитают небольшие залы, с количеством посадочных мест <60.

<a id='analyse_seat_type'></a>
## Среднее количество мест характерное для типа заведения ##

In [ ]:
fig = px.bar(data_frame=data.groupby(by='object_type')
                            .agg({'number':np.mean})
                            .reset_index()
                            .sort_values(by='number', ascending=False),
             x='object_type',
             y='number',
             template='seaborn',
             text_auto='3.0f',
             color='object_type',
             title='Среднее количество мест характерное для типа заведения',
             width=1000, height=600,
             labels={'number':'Ср. кол-во мест',
                     'object_type':'Тип заведения'})

fig.show()

## Итоги осмотра среднего количества мест характерного для типа заведения ##
1. Будем ориентироваться на заведения типа "кафе" и близкие к ним:
   - кафе ~41 посадочных мест
   - бар ~44 посадочных места
   - ресторан ~97 посадочных мест
2. Самое большое среднее количество посадочных мест предоставляет:
   - столовая

<a id='analyze_top_street'></a>
## Улицы с наибольшим количеством заведений ##

In [ ]:
data.groupby(by=['street','district']).agg({'id':'count'}).rename(columns = {'id':'count'}).sort_values(by='count', ascending=False).head(10)

In [ ]:
data

<a id='analyse_map'></a>
## Распределение заведений по территории Москвы ##

In [ ]:
# data_district = pd.read_csv(filepath_or_buffer='datasets/data_part_district.csv').drop(columns='Unnamed: 0')

In [ ]:
# fig = px.pie()
# display(data_district.groupby(by=['district'], as_index=False)
#                      .agg({'id':'count'})
#                      .rename(columns = {'id':'count'})
#                      .sort_values(by='count', ascending=False)
#                      .head(10))